In [333]:
import sqlite3
import random
import numpy as np
from faker import Faker
import pandas as pd
from datetime import date, timedelta

# Inicializar o Faker para dados em Português
faker = Faker('pt_PT')

In [334]:
Departamentos = ["Recursos Humanos", "Tecnologia da Informação", "Financeiro", "Marketing", "Vendas", "Qualidade", "Atendimento ao Cliente", "Jurídico"]
cargos = ["Gerente de Projetos", "Analista de Dados", "Desenvolvedor de Software Sénior", "Desenvolvedor de Software Júnior", "Engenheiro de DevOps","Designer UI/UX","Analista de Sistemas","Técnico de Suporte de TI","Analista de Recursos Humanos", "Especialista em Recrutamento", "Gerente de Marketing", "Especialista em Marketing Digital", "Analista de Mídia Social","Redator (Copywriter)","Gerente de Vendas","Executivo de Contas","Representante de Vendas Interno","Analista Financeiro","Contador","Gerente Financeiro","Analista de Qualidade","Inspetor de Qualidade","Agente de Atendimento ao Cliente","Supervisor de Atendimento ao Cliente",  "Especialista em Sucesso do Cliente",  "Advogado Corporativo","Assistente Jurídico","Assistente Administrativo","Recepcionista","Diretor de Operações (COO)"]

In [335]:
def criar_departamentos():
    lista_departamentos = []
    for i in range(1, len(Departamentos) + 1):
        lista_departamentos.append({
            "ID_depart": i,
            "Nome": Departamentos[i-1],
            "ID_gerente": None # Começa como Nulo
        })
    return pd.DataFrame(lista_departamentos)



In [336]:
def criar_funcionarios(n=50):
    lista_funcionarios = []
    for i in range(1, n + 1):
        lista_funcionarios.append({
            "ID_fun": i,
            "NIF": str(283684274 + i),
            "Primeiro_nome": faker.first_name(),
            "Ultimo_nome": faker.last_name(),
            "Nome_rua": faker.street_name(),
            "Nome_localidade": faker.city(),
            "Codigo_postal": faker.postcode(),
            "Num_telemovel": faker.phone_number(),
            "Email": faker.unique.email(),
            "Data_nascimento": faker.date_of_birth(minimum_age=18, maximum_age=65),
            "Cargo": random.choice([cargo for cargo in Cargos if cargo != "Gerente"]),
            "ID_depart": np.random.randint(1, len(Departamentos) + 1)
        })
    return pd.DataFrame(lista_funcionarios)

In [337]:
Departamentos = criar_departamentos()
Funcionarios = criar_funcionarios()
Departamentos["ID_gerente"] = [np.random.randint(1, len(Funcionarios) + 1) for i in range(len(Departamentos))]



In [338]:
Departamentos


,ID_depart,Nome,ID_gerente
0,1,Recursos Humanos,12
1,2,Tecnologia da Informação,25
2,3,Financeiro,17
3,4,Marketing,49
4,5,Vendas,15
5,6,Qualidade,16
6,7,Atendimento ao Cliente,45
7,8,Jurídico,32


In [339]:
Funcionarios

,ID_fun,NIF,Primeiro_nome,Ultimo_nome,Nome_rua,Nome_localidade,Codigo_postal,Num_telemovel,Email,Data_nascimento,Cargo,ID_depart
0,1,283684275,Marcos,Faria,Travessa de Santo Estêvão,Valongo,3479-687,(351) 966034026,kyara64@example.net,2003-08-28,Desenvolvedor,6
1,2,283684276,Alexandre,Nogueira,Avenida Pilar Amaral,Vila Nova de Famalicão,7516-291,(351) 926437376,gabrielmonteiro@example.com,1963-01-08,Desenvolvedor,7
2,3,283684277,Lúcia,Gonçalves,Av Luísa Fonseca,Loulé,8124-717,(351) 251506166,ucardoso@example.com,1966-01-18,Desenvolvedor,8
3,4,283684278,Valentim,Neves,Alameda de Miranda,Fiães,5505-480,969 851 752,evalente@example.com,1969-10-17,Coordenador,2
4,5,283684279,Gabriela,Sousa,Alameda de Martins,Bragança,4141-074,+351915731705,amarques@example.org,1980-04-04,Analista,2
5,6,283684280,Brian,Miranda,Alameda Petra Branco,Barcelos,4993-134,(351) 935 494 758,claudioreis@example.net,1989-03-16,Desenvolvedor,5
6,7,283684281,Fabiana,Nogueira,Travessa das Necessidades,Rebordosa,1383-912,(351) 933 250 603,liateixeira@example.net,1977-05-18,Analista,4
7,8,283684282,Vicente,Freitas,Av Jaime Pinheiro,Horta,3604-883,961127270,vitoriareis@example.net,1974-07-09,Analista,4
8,9,283684283,Letícia,Martins,Travessa de Ramos,Barreiro,9752-973,(351) 915 274 292,luis09@example.com,1999-01-18,Coordenador,7
9,10,283684284,Filipa,Coelho,Largo Assunção,Trancoso,9867-679,(351) 930 618 339,assuncaojoana@example.net,1993-07-01,Analista,8


In [340]:
# Adicionar os departamentos à tabela sem os ids dos gerentes
# conn = sqlite3.connect("C:/Users/diogo/OneDrive/Documentos/FCUL/Bases de dados/projeto_bd/Tabelas_teste.db")
# # departamentos_sem_gerente_df['ID_gerente'] = None
# departamentos_sem_gerente_df.to_sql('Departamentos', conn, if_exists='append', index=False)
# funcionarios_df.to_sql('Funcionarios', conn, if_exists='append', index=False)
# cursor = conn.cursor()
#for index, row in departamentos_df.iterrows():
  #  if pd.notna(row['ID_gerente']):
   #     cursor.execute("UPDATE Departamentos SET ID_gerente = ? WHERE ID_depart = ?", (int(row['ID_gerente']), int(row['ID_depart'])))
#conn.commit()
#conn.close()

In [341]:
def criar_remuneracoes_salarios_beneficios(funcionarios_df):
    """
    Gera dados para as tabelas Remuneracoes, Salario e Beneficios.
    A Data_fim tem 50% de chance de ser nula (contrato ativo) e 50% de chance
    de ser uma data futura.
    """
    lista_remuneracoes = []
    lista_salarios = []
    lista_beneficios = []

    tipos_de_beneficios = ['Subsídio Alimentação', 'Seguro Saúde', 'Carro Empresa', 'Subsídio Transporte', 'Telemóvel Empresa']

    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        # 1. Criar o registo na tabela Remuneracoes
        data_inicio = faker.date_between(start_date='-5y', end_date='today')
        
        # 50% de chance de o contrato ter terminado, 50% de estar ativo (None)
        # Se terminou, a data de fim é entre 3 meses e 4 anos depois do início.
        data_fim = (data_inicio + timedelta(days=random.randint(90, 1460))) if random.random() < 0.5 else None

        remuneracao_record = {
            "ID_fun": id_fun,
            "Data_inicio": data_inicio,
            "Data_fim": data_fim
        }
        lista_remuneracoes.append(remuneracao_record)
        
        # 2. Criar o registo correspondente na tabela Salario
        salario_bruto = round(np.random.normal(loc=1600, scale=700), 2)
        if salario_bruto < 950: salario_bruto = 950 # Salário mínimo aprox. para 2025

        salario_record = {
            "ID_fun": id_fun,
            "Data_inicio": data_inicio,
            "salario_bruto": salario_bruto,
        }
        lista_salarios.append(salario_record)
        
        # 3. Criar zero ou mais registos na tabela Beneficios, a chance de ter beneficios é 80%
        if random.random() < 0.8:
            num_beneficios = random.randint(1, 3)
            beneficios_para_este_fun = random.sample(tipos_de_beneficios, num_beneficios)
            
            for tipo_beneficio in beneficios_para_este_fun:
                beneficio_record = {
                    "ID_fun": id_fun,
                    "Data_inicio": data_inicio,
                    "tipo": tipo_beneficio,
                    "valor": round(random.uniform(30, 450), 2),
                }
                lista_beneficios.append(beneficio_record)

    return pd.DataFrame(lista_remuneracoes), pd.DataFrame(lista_salarios), pd.DataFrame(lista_beneficios)

In [342]:
Remuneracoes, Salarios, Beneficios = criar_remuneracoes_salarios_beneficios(Funcionarios)

In [343]:
Remuneracoes

,ID_fun,Data_inicio,Data_fim
0,1,2020-12-27,2022-01-08
1,2,2022-02-13,2023-08-08
2,3,2025-09-01,None
3,4,2022-02-12,2025-05-31
4,5,2021-02-18,None
5,6,2020-11-25,2022-09-21
6,7,2022-07-16,None
7,8,2024-02-14,None
8,9,2022-02-17,None
9,10,2024-12-25,2027-07-05


In [344]:
Salarios

,ID_fun,Data_inicio,salario_bruto
0,1,2020-12-27,1322.88
1,2,2022-02-13,950.00
2,3,2025-09-01,2226.08
3,4,2022-02-12,2141.18
4,5,2021-02-18,1542.06
5,6,2020-11-25,2396.09
6,7,2022-07-16,1777.91
7,8,2024-02-14,977.24
8,9,2022-02-17,1585.39
9,10,2024-12-25,950.00


In [345]:
Beneficios

,ID_fun,Data_inicio,tipo,valor
0,1,2020-12-27,Subsídio Transporte,394.77
1,2,2022-02-13,Seguro Saúde,114.42
2,5,2021-02-18,Seguro Saúde,443.93
3,5,2021-02-18,Subsídio Transporte,158.39
4,6,2020-11-25,Subsídio Transporte,64.18
...,...,...,...,...
76,48,2023-08-29,Subsídio Alimentação,276.53
77,48,2023-08-29,Subsídio Transporte,147.87
78,49,2021-10-08,Seguro Saúde,108.20
79,49,2021-10-08,Carro Empresa,234.10


In [346]:
def criar_ferias(funcionarios_df):
    """
    NÃO TEM O NUM_DIAS_PERMITIDOS
    Gera um histórico de férias para os funcionários.
    Cada funcionário pode ter entre 0 a 4 registos de férias.
    """
    lista_ferias = []
    estados_possiveis = ['Aprovado', 'Aprovado', 'Aprovado', 'Aprovado', 'Rejeitado', 'Por aprovar']
    
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        num_periodos_ferias = random.randint(0, 4)
        
        for _ in range(num_periodos_ferias):
            data_inicio = faker.date_between(start_date='-3y', end_date='today')
            num_dias = random.randint(2, 15)
            data_fim = data_inicio + timedelta(days=num_dias)
            
            ferias_record = {
                "ID_fun": id_fun,
                "data_inicio": data_inicio,
                "data_fim": data_fim,
                "num_dias": num_dias,
                "estado_aprov": random.choice(estados_possiveis)
            }
            lista_ferias.append(ferias_record)
            
    return pd.DataFrame(lista_ferias)

In [347]:
Ferias = criar_ferias(Funcionarios)
Ferias

,ID_fun,data_inicio,data_fim,num_dias,estado_aprov
0,1,2025-04-16,2025-04-24,8,Aprovado
1,1,2023-02-06,2023-02-09,3,Por aprovar
2,2,2023-02-26,2023-03-12,14,Aprovado
3,2,2024-12-21,2025-01-05,15,Aprovado
4,2,2025-10-09,2025-10-14,5,Aprovado
...,...,...,...,...,...
88,49,2024-05-08,2024-05-17,9,Aprovado
89,49,2024-03-29,2024-04-13,15,Aprovado
90,49,2025-07-25,2025-08-07,13,Aprovado
91,49,2025-07-06,2025-07-20,14,Aprovado


In [348]:
def criar_dependentes(funcionarios_df):
    """
    Gera uma lista de dependentes e depois limpa os dados para garantir que
    nenhum funcionário tem mais do que um dependente com o parentesco 'Pai/Mãe'.
    """
    lista_dependentes = []
    # Usamos 'Pai/Mãe' como uma única categoria para simplificar
    parentescos_possiveis = ['Filho(a)', 'Cônjuge', 'Enteado(a)', 'Pai/Mãe', 'Filho(a)'] # Repetimos 'Filho(a)' para aumentar a probabilidade
    sexos_possiveis = ['Masculino', 'Feminino', 'Outro']
    
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        if random.random() < 0.7:
            num_dependentes = random.randint(1, 4) # Aumentei para 4 para ter mais chance de criar duplicados para limpar
            
            for _ in range(num_dependentes):
                dependente_record = {
                    "ID_Fun": id_fun,
                    "nome": faker.name(),
                    "sexo": random.choice(sexos_possiveis),
                    "data_nascimento": faker.date_of_birth(minimum_age=0, maximum_age=80),
                    "parentesco": random.choice(parentescos_possiveis)
                }
                lista_dependentes.append(dependente_record)
                
    # --- PASSO DE LIMPEZA ---

    # 1. Criar o DataFrame com os dados
    if not lista_dependentes:
        return pd.DataFrame() # Retorna um DataFrame vazio se nenhum dependente foi gerado
        
    dependentes_bruto_df = pd.DataFrame(lista_dependentes)
    
    # 2. Separar os pais/mães dos outros dependentes
    pais_maes_df = dependentes_bruto_df[dependentes_bruto_df['parentesco'] == 'Pai/Mãe']
    outros_dependentes_df = dependentes_bruto_df[dependentes_bruto_df['parentesco'] != 'Pai/Mãe']
    
    # 3. Remover duplicados APENAS da parte dos pais/mães
    # drop_duplicates mantém a primeira ocorrência de cada combinação de 'ID_Fun' e 'parentesco'
    pais_maes_unicos_df = pais_maes_df.drop_duplicates(subset=['ID_Fun', 'parentesco'], keep='first')
    
    # 4. Juntar novamente as duas partes para ter o DataFrame final e limpo
    dependentes_final_df = pd.concat([pais_maes_unicos_df, outros_dependentes_df], ignore_index=True)
    
    return dependentes_final_df

In [349]:
Dependentes = criar_dependentes(Funcionarios)
Dependentes

,ID_Fun,nome,sexo,data_nascimento,parentesco
0,1,Beatriz Paiva-Sá,Outro,1956-08-21,Pai/Mãe
1,4,Joana Brito-Loureiro,Feminino,2009-10-20,Pai/Mãe
2,5,Salomé Antunes,Feminino,1973-07-30,Pai/Mãe
3,8,Vicente Araújo,Outro,1961-02-14,Pai/Mãe
4,11,Érica Campos,Outro,1972-11-14,Pai/Mãe
...,...,...,...,...,...
74,41,Eduarda do Amaral,Outro,1965-04-07,Cônjuge
75,41,Matias Pereira,Outro,2020-10-22,Filho(a)
76,46,Miguel Gomes,Masculino,1998-12-17,Cônjuge
77,46,Patrícia Almeida-Reis,Outro,1992-03-24,Cônjuge


In [350]:
def criar_faltas(funcionarios_df):
    """
    Gera um histórico de faltas para os funcionários.
    Cada funcionário terá entre 0 e 10 faltas registadas.
    """
    lista_faltas = []
    justificacoes_comuns = ["Consulta médica","Doença súbita","Apoio à família","Assuntos pessoais urgentes","Procedimentos legais/burocráticos","Avaria de transporte", None, None] # Para simular faltas injustificadas

    # Itera sobre cada funcionário para criar o seu histórico de faltas
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        # Cada funcionário terá um número aleatório de faltas (de 0 a 10)
        num_faltas = int(np.random.normal(loc=3, scale=2))
        
        # Para evitar que as faltas do mesmo funcionário sejam todas no mesmo dia,
        # geramos um conjunto de datas únicas para este funcionário
        datas_possiveis = [faker.date_between(start_date='-3y', end_date='today') for _ in range(num_faltas)]
        datas_unicas = set(datas_possiveis) # 'set' remove datas duplicadas automaticamente
        
        for data_falta in datas_unicas:
            falta_record = {
                "ID_Fun": id_fun,
                "Data": data_falta,
                "Justificacao": random.choice(justificacoes_comuns)
            }
            lista_faltas.append(falta_record)
            
    return pd.DataFrame(lista_faltas)

In [351]:
Faltas = criar_faltas(Funcionarios)
Faltas

,ID_Fun,Data,Justificacao
0,1,2025-05-16,Apoio à família
1,1,2023-02-02,Consulta médica
2,1,2025-07-14,Avaria de transporte
3,1,2024-05-20,Doença súbita
4,1,2023-11-21,None
...,...,...,...
131,48,2024-07-31,Apoio à família
132,48,2023-05-16,Procedimentos legais/burocráticos
133,49,2025-09-23,Consulta médica
134,49,2023-01-22,Avaria de transporte


In [352]:
def criar_historico_empresas(funcionarios_df):
    """
    Gera um histórico profissional para os funcionários com nomes de departamento realistas.
    As datas são geradas de forma cronológica invertida para garantir consistência.
    """
    lista_historico = []

    # Lista de nomes de departamentos genéricos para empresas anteriores.
    departamentos_genericos = [
        'Marketing', 'Vendas', 'Tecnologia da Informação', 'Recursos Humanos',
        'Financeiro', 'Operações', 'Logística', 'Suporte ao Cliente',
        'Produto', 'Engenharia', 'Qualidade', 'Administrativo'
    ]
    
    # Itera sobre cada funcionário
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        # 80% de chance de o funcionário ter um histórico profissional
        if random.random() < 0.8:
            num_historicos = random.randint(1, 3)
            
            latest_end_date = date.today() - timedelta(days=random.randint(90, 730))
            
            for i in range(num_historicos):
                data_fim = latest_end_date
                data_inicio = data_fim - timedelta(days=random.randint(365, 365 * 5))
                
                historico_record = {
                    "ID_Fun": id_fun,
                    "Nome_empresa": faker.company(),
                    # Agora escolhemos um departamento aleatório da lista
                    "Nome_departamento": random.choice(departamentos_genericos),
                    "Cargo": faker.job(),
                    "Data_inicio": data_inicio,
                    "Data_fim": data_fim
                }
                lista_historico.append(historico_record)
                
                latest_end_date = data_inicio - timedelta(days=random.randint(30, 365))

    return pd.DataFrame(lista_historico)

In [353]:
Historico_empresas = criar_historico_empresas(Funcionarios)
Historico_empresas

,ID_Fun,Nome_empresa,Nome_departamento,Cargo,Data_inicio,Data_fim
0,1,Vaz,Logística,Disc Jockey,2020-12-27,2023-12-28
1,2,Marques,Engenharia,Trabalhador não qualificado das minas,2023-02-21,2024-09-07
2,2,Maia,Recursos Humanos,Técnico em redes e sistemas de computadores,2020-12-03,2022-09-22
3,3,Pinto S/A,Vendas,Artesão de artigos em madeira,2022-12-18,2024-11-27
4,4,Valente,Administrativo,Estofador,2022-09-10,2023-12-21
...,...,...,...,...,...,...
73,48,Moura Guerreiro Lda.,Operações,Operador de instalações e máquinas para outros...,2022-02-27,2025-01-11
74,49,Matos,Marketing,"Outro pessoal de apoio de tipo administrativo,...",2023-10-09,2025-06-18
75,49,Mota,Logística,"Rececionista, exceto de hotel",2021-03-17,2023-09-04
76,50,Ribeiro,Engenharia,Engenheiro de obras de engenharia civil,2022-09-21,2025-05-16


In [354]:
def criar_modulo_recrutamento(funcionarios_df, departamentos_df, num_candidatos=40, num_vagas=10):
    """
    Gera dados para as tabelas Candidatos, Vagas, Requisitos_vaga e Candidato_a.
    """
    lista_candidatos = []
    lista_vagas = []
    lista_requisitos = []
    lista_candidaturas = []

    # --- 1. Gerar Candidatos ---
    for i in range(1, num_candidatos + 1):
        candidato_record = {
            "ID_cand": i,
            "Nome": faker.name(),
            "Email": faker.unique.email(),
            "Telemovel": faker.phone_number(),
            "CV": None, # BLOBs são representados como None
            "Carta_motivação": None
        }
        lista_candidatos.append(candidato_record)
    candidatos_df = pd.DataFrame(lista_candidatos)
    ids_candidatos_disponiveis = candidatos_df['ID_cand'].tolist()

    # --- 2. Gerar Vagas ---
    ids_departamentos_disponiveis = departamentos_df['ID_depart'].tolist()
    estados_vaga = ['Aberta', 'Fechada', 'Fechada', 'Fechada', 'Suspensa'] # Mais chance de estar fechada

    for i in range(1, num_vagas + 1):
        vaga_record = {
            "ID_vaga": i,
            "Data_abertura": faker.date_between(start_date='-2y', end_date='today'),
            "Estado": random.choice(estados_vaga),
            "ID_depart": random.choice(ids_departamentos_disponiveis)
        }
        lista_vagas.append(vaga_record)
    vagas_df = pd.DataFrame(lista_vagas)
    
    # --- 3. Gerar Requisitos para cada Vaga ---
    requisitos_possiveis = [
        'Python', 'SQL', 'Gestão de Projetos', 'Comunicação', 'Inglês Fluente',
        'Análise de Dados', 'React', 'Liderança', 'Trabalho em Equipa', 'Excel Avançado',
        'Marketing Digital', 'Contabilidade', 'Java', 'Atendimento ao Cliente'
    ]
    for index, vaga in vagas_df.iterrows():
        # Cada vaga terá entre 2 e 5 requisitos
        num_requisitos = random.randint(2, 5)
        # Usamos random.sample para garantir que os requisitos para a mesma vaga são únicos
        requisitos_para_vaga = random.sample(requisitos_possiveis, num_requisitos)
        
        for req in requisitos_para_vaga:
            requisito_record = {
                "ID_vaga": vaga['ID_vaga'],
                "Requisito": req
            }
            lista_requisitos.append(requisito_record)
    requisitos_df = pd.DataFrame(lista_requisitos)

    # --- 4. Gerar Candidaturas (Candidato_a) ---
    # Para ser mais realista, vamos pegar nos IDs dos funcionários de RH para serem os recrutadores
    id_depart_rh = departamentos_df[departamentos_df['Nome'] == 'Recursos Humanos']['ID_depart'].iloc[0]
    ids_recrutadores = funcionarios_df[funcionarios_df['ID_depart'] == id_depart_rh]['ID_fun'].tolist()
    # Fallback: se não houver ninguém em RH, usamos qualquer funcionário
    if not ids_recrutadores:
        ids_recrutadores = funcionarios_df['ID_fun'].tolist()
        
    estados_candidatura = ['Submetido', 'Em análise', 'Entrevista', 'Rejeitado', 'Rejeitado', 'Rejeitado', 'Contratado']

    # Cada candidato vai candidatar-se a um número aleatório de vagas (0 a 3)
    for id_candidato in ids_candidatos_disponiveis:
        num_candidaturas = random.randint(0, 3)
        if num_candidaturas > 0:
            # Escolhe 'num_candidaturas' vagas aleatórias e únicas para se candidatar
            vagas_aplicadas = vagas_df.sample(num_candidaturas)
            
            for index, vaga in vagas_aplicadas.iterrows():
                # A data da candidatura tem de ser posterior à data de abertura da vaga
                data_candidatura = faker.date_between(start_date=vaga['Data_abertura'], end_date='today')
                
                candidatura_record = {
                    "ID_cand": id_candidato,
                    "ID_Vaga": vaga['ID_vaga'],
                    "Data_cand": data_candidatura,
                    "Estado": random.choice(estados_candidatura),
                    "ID_recrutador": random.choice(ids_recrutadores)
                }
                lista_candidaturas.append(candidatura_record)
    candidato_a_df = pd.DataFrame(lista_candidaturas)
    
    return candidatos_df, vagas_df, requisitos_df, candidato_a_df

In [355]:
Candidatos, Vagas, Requisitos_vaga, Candidato_a = criar_modulo_recrutamento(Funcionarios, Departamentos)

In [356]:
Candidatos

,ID_cand,Nome,Email,Telemovel,CV,Carta_motivação
0,1,Tomé Figueiredo,rfreitas@example.org,963165661,None,None
1,2,Mélanie Gomes-Amorim,hmonteiro@example.net,(351) 923000153,None,None
2,3,Ivo Monteiro,ycastro@example.com,(351) 913 701 649,None,None
3,4,Mariana Carneiro,erikafernandes@example.com,923 304 877,None,None
4,5,Filipe Morais-Abreu,sofiacardoso@example.net,(351) 934774934,None,None
5,6,César Leite,raquel83@example.com,+351929290161,None,None
6,7,Irina Sousa,zferreira@example.net,+351938203353,None,None
7,8,Rafaela-Pilar Torres,enzogomes@example.net,(351) 919605038,None,None
8,9,Benedita-Kyara Assunção,larafreitas@example.com,(351) 926 935 557,None,None
9,10,Joana Machado,micaelsilva@example.org,(351) 920669154,None,None


In [357]:
Vagas

,ID_vaga,Data_abertura,Estado,ID_depart
0,1,2025-08-30,Suspensa,7
1,2,2025-06-09,Fechada,5
2,3,2024-11-02,Aberta,2
3,4,2024-10-03,Suspensa,6
4,5,2024-07-09,Fechada,2
5,6,2024-01-29,Fechada,5
6,7,2023-12-23,Fechada,6
7,8,2024-03-21,Fechada,8
8,9,2024-02-11,Fechada,3
9,10,2025-01-29,Fechada,5


In [358]:
Requisitos_vaga


,ID_vaga,Requisito
0,1,Inglês Fluente
1,1,SQL
2,1,Atendimento ao Cliente
3,1,Trabalho em Equipa
4,1,Liderança
5,2,Inglês Fluente
6,2,Análise de Dados
7,3,Python
8,3,Liderança
9,3,Contabilidade


In [359]:
Candidato_a

,ID_cand,ID_Vaga,Data_cand,Estado,ID_recrutador
0,1,7,2024-04-29,Rejeitado,11
1,1,5,2025-03-02,Em análise,25
2,1,4,2025-05-20,Submetido,23
3,2,3,2025-01-30,Contratado,43
4,2,6,2025-10-11,Rejeitado,32
5,2,2,2025-08-25,Entrevista,11
6,3,2,2025-09-14,Entrevista,32
7,3,6,2025-04-17,Rejeitado,23
8,4,4,2025-05-28,Rejeitado,30
9,4,7,2025-07-06,Entrevista,25


In [360]:
def criar_catalogo_formacoes():
    """
    Cria um catálogo fixo e manual de 5 formações com descrições realistas.
    """
    dados_formacoes = [
        {"ID_for": 1, "Nome_formacao": 'Liderança e Gestão de Equipas', "Descricao": 'Desenvolver competências de liderança, motivação e gestão de conflitos para chefes de equipa.', "Data_inicio": date(2023, 10, 5), "Data_fim": date(2023, 10, 10), "Estado": 'Concluida'},
        {"ID_for": 2, "Nome_formacao": 'Cibersegurança para Colaboradores', "Descricao": 'Sensibilizar para as melhores práticas de segurança digital, phishing e proteção de dados.', "Data_inicio": date(2024, 3, 15), "Data_fim": date(2024, 3, 16), "Estado": 'Concluida'},
        {"ID_for": 3, "Nome_formacao": 'Análise de Dados com Python', "Descricao": 'Introdução à análise de dados utilizando as bibliotecas Pandas e Matplotlib.', "Data_inicio": date(2024, 9, 2), "Data_fim": date(2024, 9, 13), "Estado": 'Em curso'},
        {"ID_for": 4, "Nome_formacao": 'Técnicas de Negociação e Vendas', "Descricao": 'Aperfeiçoar técnicas de comunicação, persuasão e fecho de negócios para a equipa comercial.', "Data_inicio": date(2025, 2, 10), "Data_fim": date(2025, 2, 14), "Estado": 'Planeada'},
        {"ID_for": 5, "Nome_formacao": 'Inteligência Emocional no Trabalho', "Descricao": 'Gerir emoções, melhorar o autoconhecimento e a empatia nas relações interpessoais no trabalho.', "Data_inicio": date(2025, 5, 19), "Data_fim": date(2025, 5, 20), "Estado": 'Planeada'}
    ]
    
    return pd.DataFrame(dados_formacoes)

In [361]:
Formacoes = criar_catalogo_formacoes()
Formacoes

,ID_for,Nome_formacao,Descricao,Data_inicio,Data_fim,Estado
0,1,Liderança e Gestão de Equipas,"Desenvolver competências de liderança, motivaç...",2023-10-05,2023-10-10,Concluida
1,2,Cibersegurança para Colaboradores,Sensibilizar para as melhores práticas de segu...,2024-03-15,2024-03-16,Concluida
2,3,Análise de Dados com Python,Introdução à análise de dados utilizando as bi...,2024-09-02,2024-09-13,Em curso
3,4,Técnicas de Negociação e Vendas,"Aperfeiçoar técnicas de comunicação, persuasão...",2025-02-10,2025-02-14,Planeada
4,5,Inteligência Emocional no Trabalho,"Gerir emoções, melhorar o autoconhecimento e a...",2025-05-19,2025-05-20,Planeada


In [362]:
def criar_teve_formacao(funcionarios_df, formacoes_df):
    """
    Gera as inscrições dos funcionários nas formações de forma aleatória e simples,
    sem verificar o estado ou as datas das formações.
    """
    lista_teve_formacao = []

    # Se não houver formações no catálogo, não podemos fazer nada.
    if formacoes_df.empty:
        return pd.DataFrame()

    # Itera sobre cada funcionário
    for index, funcionario in funcionarios_df.iterrows():
        # 60% de chance de o funcionário ter tido alguma formação
        if random.random() < 0.6:
            
            # Número de formações que este funcionário teve (1 ou 2)
            num_formacoes_feitas = random.randint(1, 2)
            
            # Escolhe aleatoriamente formações do catálogo COMPLETO
            formacoes_a_inscrever = formacoes_df.sample(num_formacoes_feitas)
            
            for i, formacao in formacoes_a_inscrever.iterrows():
                teve_formacao_record = {
                    "ID_Fun": funcionario['ID_fun'],
                    "ID_for": formacao['ID_for'],
                    "Certificado": None, # BLOB
                    "Data_inicio": formacao['Data_inicio'], # Copia as datas diretamente
                    "Data_fim": formacao['Data_fim']
                }
                lista_teve_formacao.append(teve_formacao_record)

    return pd.DataFrame(lista_teve_formacao)

In [363]:
Teve_Formacao = criar_teve_formacao(Funcionarios, Formacoes)
Teve_Formacao

,ID_Fun,ID_for,Certificado,Data_inicio,Data_fim
0,4,3,None,2024-09-02,2024-09-13
1,7,5,None,2025-05-19,2025-05-20
2,7,4,None,2025-02-10,2025-02-14
3,8,2,None,2024-03-15,2024-03-16
4,8,5,None,2025-05-19,2025-05-20
5,11,3,None,2024-09-02,2024-09-13
6,11,1,None,2023-10-05,2023-10-10
7,13,3,None,2024-09-02,2024-09-13
8,13,4,None,2025-02-10,2025-02-14
9,14,5,None,2025-05-19,2025-05-20


In [364]:
FRASES_CRITERIOS = [
    "Avaliação da pontualidade e assiduidade no último semestre.",
    "Desempenho nos projetos de equipa e capacidade de colaboração.",
    "Capacidade de resolução de problemas e tomada de decisão.",
    "Qualidade da comunicação com o cliente e stakeholders internos.",
    "Iniciativa e proatividade na sugestão de novas melhorias.",
    "Qualidade técnica do trabalho entregue e atenção ao detalhe.",
    "Gestão de tempo, cumprimento de prazos e organização."
]

FRASES_AUTOAVALIACAO = [
    "Sinto que atingi todos os meus objetivos principais este semestre.",
    "Completei a formação X e sinto-me mais confiante nas minhas tarefas.",
    "Gostaria de ter mais formação na área de análise de dados para contribuir melhor.",
    "Consegui melhorar a minha organização e gestão de tempo, como foi discutido.",
    "Tive alguma dificuldade em gerir o projeto Y, mas aprendi muito com o processo.",
    "Preciso de focar-me em ser mais conciso nas reuniões de equipa."
]


def criar_avaliacoes(funcionarios_df, num_avaliacoes=40):
    """
    Gera um número fixo de avaliações (ex: 20) com frases realistas
    e IDs de avaliadores inteiros.
    """
    lista_avaliacoes = []
    
    # Obter uma lista de todos os IDs de funcionários
    ids_funcionarios_disponiveis = funcionarios_df['ID_fun'].tolist()

    # Se não houver funcionários, não podemos fazer nada
    if not ids_funcionarios_disponiveis or len(ids_funcionarios_disponiveis) < 2:
        print("Aviso: É preciso ter pelo menos 2 funcionários para gerar avaliações.")
        return pd.DataFrame()

    for _ in range(num_avaliacoes):
        
        # 1. Escolhe um funcionário para ser AVALIADO
        id_fun_avaliado = random.choice(ids_funcionarios_disponiveis)
        
        # 2. Escolhe um AVALIADOR
        id_avaliador = random.choice(ids_funcionarios_disponiveis)
        
        # 3. Garante que não é a mesma pessoa (autoavaliação é noutro campo)
        while id_avaliador == id_fun_avaliado:
            id_avaliador = random.choice(ids_funcionarios_disponiveis)
            
        # 4. Gera uma data única para a chave primária
        data_avaliacao = faker.date_between(start_date='-3y', end_date='today')

        # 5. 70% de chance de ter uma autoavaliação
        autoavaliacao = random.choice(FRASES_AUTOAVALIACAO) if random.random() < 0.7 else None
        
        avaliacao_record = {
            "ID_Fun": id_fun_avaliado,
            "ID_avaliador": id_avaliador, # Isto será sempre um INT
            "Data": data_avaliacao,
            "Avaliacao": None, # BLOB
            "Criterios": random.choice(FRASES_CRITERIOS), # Frase realista
            "Autoavaliacao": autoavaliacao # Frase realista ou None
        }
        
        lista_avaliacoes.append(avaliacao_record)

    # Criar o DataFrame
    avaliacoes_df = pd.DataFrame(lista_avaliacoes)

    # Salvaguarda: Remove duplicados na chave primária (caso tenhamos gerado o mesmo trio por acaso)
    avaliacoes_df = avaliacoes_df.drop_duplicates(subset=['ID_Fun', 'ID_avaliador', 'Data'], keep='first')
    return avaliacoes_df

In [365]:
Avaliacoes = criar_avaliacoes(Funcionarios)
Avaliacoes

,ID_Fun,ID_avaliador,Data,Avaliacao,Criterios,Autoavaliacao
0,48,15,2024-04-06,None,Iniciativa e proatividade na sugestão de novas...,Gostaria de ter mais formação na área de análi...
1,48,18,2023-06-21,None,Qualidade técnica do trabalho entregue e atenç...,None
2,6,20,2023-09-10,None,Avaliação da pontualidade e assiduidade no últ...,Preciso de focar-me em ser mais conciso nas re...
3,14,36,2025-02-13,None,Desempenho nos projetos de equipa e capacidade...,None
4,7,12,2024-06-06,None,Iniciativa e proatividade na sugestão de novas...,Consegui melhorar a minha organização e gestão...
5,49,25,2023-08-28,None,Avaliação da pontualidade e assiduidade no últ...,Gostaria de ter mais formação na área de análi...
6,32,39,2024-01-30,None,Desempenho nos projetos de equipa e capacidade...,None
7,7,38,2024-09-17,None,Iniciativa e proatividade na sugestão de novas...,None
8,18,27,2024-07-23,None,Avaliação da pontualidade e assiduidade no últ...,Sinto que atingi todos os meus objetivos princ...
9,38,22,2023-08-01,None,Capacidade de resolução de problemas e tomada ...,Consegui melhorar a minha organização e gestão...


In [366]:
def criar_utilizadores_permissoes(funcionarios_df):
    """
    Gera dados para as tabelas Utilizadores e Permissoes.
    Cria um utilizador para cada funcionário e atribui permissões aleatórias.
    """
    lista_utilizadores = []
    lista_permissoes = []
    
    # A sua lista de permissões válidas
    permissoes_possiveis = ['Funcionario', 'Manager', 'RH', 'Admin', 'Financeiro']

    # Itera sobre CADA funcionário para criar uma conta de utilizador
    for index, funcionario in funcionarios_df.iterrows():
        id_fun = funcionario['ID_fun']
        
        # --- 1. Criar o registo na tabela Utilizadores ---
        utilizador_record = {
            "ID_Fun": id_fun,
            # Gera uma password aleatória. 'special_chars=False' para caber no VARCHAR(30)
            "Password": faker.password(length=12, special_chars=False) 
        }
        lista_utilizadores.append(utilizador_record)
        
        # --- 2. Criar os registos na tabela Permissoes ---
        
        # Cada utilizador terá entre 1 e 3 permissões
        num_permissoes = random.randint(1, 3)
        
        # Escolhe 'num_permissoes' permissões únicas da lista
        permissoes_para_este_user = random.sample(permissoes_possiveis, num_permissoes)
        
        # Cria um registo para cada permissão atribuída
        for perm in permissoes_para_este_user:
            permissao_record = {
                "ID_Fun": id_fun,
                "Permissao": perm
            }
            lista_permissoes.append(permissao_record)

    utilizadores_df = pd.DataFrame(lista_utilizadores)
    permissoes_df = pd.DataFrame(lista_permissoes)
    
    return utilizadores_df, permissoes_df

In [367]:
Utilizadores, Permissoes = criar_utilizadores_permissoes(Funcionarios)

In [368]:
Utilizadores

,ID_Fun,Password
0,1,bDpTAui61Dr1
1,2,T83Uo8vSB3oG
2,3,Vt3tInx9Mnfl
3,4,HQT73jr9mZ9n
4,5,89GignuJr02D
5,6,QD8P4KvAKB1u
6,7,5juPAsu54V2q
7,8,8IWSgkbwPVIf
8,9,wVZYLQhe62pU
9,10,0ydcXVuzVTLJ


In [369]:
Permissoes

,ID_Fun,Permissao
0,1,Admin
1,1,RH
2,2,Funcionario
3,2,Manager
4,2,Financeiro
...,...,...
97,48,Manager
98,49,RH
99,49,Admin
100,49,Financeiro


In [370]:
import pandas as pd
from datetime import date, datetime
import math

print("A iniciar a geração do ficheiro 'dados_insercao.sql' (um INSERT por tabela)...")

# --- Função Auxiliar (sem alterações) ---
def formatar_valor_sql(valor):
    if pd.isna(valor) or valor is None:
        return "NULL"
    if isinstance(valor, str):
        return f"'{valor.replace("'", "''")}'"
    if isinstance(valor, (int, float)):
        return str(valor)
    if isinstance(valor, (date, datetime)):
        return f"'{valor.isoformat()}'"
    return f"'{str(valor)}'"

# --- Dicionário com todos os DataFrames (sem alterações) ---
todos_dataframes = {
    "Remuneracoes": Remuneracoes,
    "Salario": Salarios,
    "Beneficios": Beneficios,
    "Ferias": Ferias,
    "Dependentes": Dependentes,
    "Faltas": Faltas,
    "Historico_empresas": Historico_empresas,
    "Candidatos": Candidatos,
    "Vagas": Vagas,
    "Candidato_a": Candidato_a,
    "Requisitos_vaga": Requisitos_vaga,
    "Formacoes": Formacoes,
    "Teve_Formacao": Teve_Formacao,
    "Avaliacoes": Avaliacoes,
    "Utilizadores": Utilizadores,
    "Permissoes": Permissoes
}

# --- NOVO: Função para gerar UM ÚNICO INSERT por tabela ---
def escrever_insert_unico_sql(f, df, nome_tabela, colunas_sql=None):
    """
    Escreve um DataFrame num ficheiro SQL usando UMA ÚNICA instrução INSERT.
    """
    if df.empty:
        f.write(f"\n-- Tabela {nome_tabela} está vazia. A saltar.\n")
        return

    f.write(f"\n-- Dados para {nome_tabela}\n")
    
    if colunas_sql:
        cols_sql = colunas_sql
    else:
        cols_sql = ", ".join(df.columns)
    
    # Escreve o início da instrução
    f.write(f"INSERT INTO {nome_tabela} ({cols_sql}) VALUES\n")
    
    linhas_de_valores = []
    # Itera por TODAS as linhas do DataFrame
    for row in df.itertuples(index=False, name=None):
        valores_formatados = f"({', '.join([formatar_valor_sql(val) for val in row])})"
        linhas_de_valores.append(valores_formatados)
    
    # Junta todas as linhas de valores com uma vírgula
    f.write(",\n".join(linhas_de_valores))
    f.write(";\n") # Termina a instrução INSERT

# --- Processo de Escrita do Ficheiro (Atualizado) ---
try:
    with open('dados_insercao.sql', 'w', encoding='utf-8') as f:
        f.write("-- Script de Inserção de Dados Gerado Automaticamente\n")
        f.write("BEGIN TRANSACTION;\n")

        # 1. Departamentos (Inserir com gerente NULL)
        df_deptos_temp = Departamentos[['ID_depart', 'Nome']].copy()
        df_deptos_temp['ID_gerente'] = None
        escrever_insert_unico_sql(f, df_deptos_temp, "Departamentos", colunas_sql="ID_depart, Nome, ID_gerente")
        
        # 2. Funcionarios
        escrever_insert_unico_sql(f, Funcionarios, "Funcionarios")

        # 3. UPDATE Departamentos (Agora com os gerentes)
        f.write("\n-- 3. Atualização dos Gerentes nos Departamentos\n")
        for row in Departamentos.itertuples(index=False):
            if pd.notna(row.ID_gerente):
                f.write(f"UPDATE Departamentos SET ID_gerente = {int(row.ID_gerente)} WHERE ID_depart = {row.ID_depart};\n")

        # 4. Inserção de todas as outras tabelas
        f.write("\n-- 4. Inserção das restantes tabelas\n")
        for nome_tabela, df in todos_dataframes.items():
            
            # Lógica especial para Candidato_a para renomear colunas
            if nome_tabela == 'Candidato_a':
                escrever_insert_unico_sql(f, df, nome_tabela, colunas_sql="ID_cand, ID_Vaga, Data_cand, Estado, ID_recrutador")
            else:
                escrever_insert_unico_sql(f, df, nome_tabela)

        f.write("\nCOMMIT;\n")

    print("\n--- SUCESSO! ---")
    print("O ficheiro 'dados_insercao.sql' foi gerado com sucesso.")
    print("Cada tabela tem agora UM ÚNICO comando INSERT com todos os seus dados.")

except Exception as e:
    print(f"\n--- ERRO ---")
    print(f"Ocorreu um erro ao gerar o ficheiro: {e}")
    print("Verifique os nomes dos seus DataFrames.")

A iniciar a geração do ficheiro 'dados_insercao.sql' (um INSERT por tabela)...

--- SUCESSO! ---
O ficheiro 'dados_insercao.sql' foi gerado com sucesso.
Cada tabela tem agora UM ÚNICO comando INSERT com todos os seus dados.
